## downloading the data set

In [ ]:
# note: i'll fix this later i was having issues w the dataloader and kaggle file so i just directly uploaded them here
"""
Image Patching - transforming an image into a sequence 
1. we patch it in a 16x16 pixel tiles
2. we pass each of the comopents trough the first component of the transformer encoder - gives us embedding
3. red bars - rep the vectors [the blocks are just fully connected nn]

most applications use einops - to ge the input for transformer to reshape 

CLS token - using for classification - bc the transformer is sequence to sequenxe 

positional embedding - make the model understand where each patch is placed in the orginal image - they are vectors and distance is constant 

attention and transformer stuff 
multi head is scale dot product -theres a pytorch implementation

normalization - layer normalization 

feed forward - linear head 

residulas - to improve the flow 
""""
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"priyanayak26","key":"3906c6ccce8f2ff273b867baa7a7ec58"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d xhlulu/leafsnap-dataset
!unzip leafsnap-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-03-2.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-03-3.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-03-4.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-04-1.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-04-2.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-04-3.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-04-4.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-05-1.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-05-2.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-05-3.png  
  inflating: leafsnap-dataset/dataset/segmented/lab/quercus_cerris/ny1145-05-4.png  
  inflating: l

# Set up the data loader

creating the dataset and dataloader

In [3]:
# it wasnt working so im manually uploading it
from google.colab import files
files.upload()

Saving dataloader.py to dataloader.py


{'dataloader.py': b'import os\r\nimport numpy as np\r\nimport pandas as pd\r\nfrom PIL import Image\r\nfrom torch.utils.data import Dataset\r\n\r\n\r\nclass LeafsnapDataset(Dataset):\r\n    def __init__(self, image_path, root_folder, use_segmented=False, source="both", transform=None):\r\n        self.root_directory = root_folder\r\n        self.transform = transform\r\n\r\n        self.image_info = pd.read_csv(image_path, sep=\'\\t\')\r\n\r\n        if source != "both":\r\n            self.image_info = self.image_info[self.image_info[\'source\'] == source]\r\n            self.image_info = self.image_info.reset_index(drop=True)\r\n\r\n        if use_segmented:\r\n            self.img_path_col = \'segmented_path\'\r\n        else:\r\n            self.img_path_col = \'image_path\'\r\n\r\n        label_hashmap = {}\r\n        species_set = set(sorted(self.image_info[\'species\'].unique()))\r\n\r\n        for i, species in enumerate(species_set):\r\n            label_hashmap[species] = i\r

imports

In [4]:
from dataloader import *

import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# from dataloader import LeafsnapDataset

define image transformations

In [5]:
# transformer
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# building data loader

inialize the data set

In [6]:
root_directory = "/content/leafsnap-dataset/"
image_paths_file = root_directory + "leafsnap-dataset-images.txt"

dataset = LeafsnapDataset(image_paths_file, root_directory, use_segmented=False, transform=transform)

build the dataloader

In [7]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

verifying that the dataloader is workin [debugging to see what it shows]


In [8]:
data_iter = iter(dataloader)
images, labels = next(data_iter)
print(f"Batch size: {images.shape}, Labels: {labels}")

Batch size: torch.Size([32, 3, 224, 224]), Labels: tensor([129,  32,  54,  67,  72, 102,  48,  92, 108, 119, 136,  83, 109,  25,
        123,   1, 133,  14, 136, 151, 139, 119, 169,  32,  14,  23,   1,  22,
         47,   5, 171,  62])


## Vision Transformer (ViT) implementation


install needed libraries

In [ ]:
!pip install torch torchvision timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

load pre-trained ViT model

In [9]:
import timm
import torch.nn as nn
## defining the vision transformer
class ViTLeafClassifier(nn.Module):
    def __init__(self, num_classes=185):
        super(ViTLeafClassifier, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)  # Load ViT with pre-trained weights
        self.vit.head = nn.Linear(self.vit.head.in_features, num_classes)  # Modify classifier for 185 leaf classes

    def forward(self, x):
        return self.vit(x)

# Train the model

In [10]:
# i have trained it like 3 times already and it randomly crashes or my laptop does, i will finish training in a bit and push again ik colab hates me rn
# training loop
# Define the device
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, loss function, and optimizer
model = ViTLeafClassifier(num_classes=185).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

# Training loop
num_epochs = 10  # Adjust as needed
print(f"\n Starting Training on {device}\n")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    print(f"\n Epoch {epoch+1}/{num_epochs}")

    for batch_idx, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Track accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Print updates every 10 batches
        if (batch_idx + 1) % 10 == 0 or batch_idx == 0:
            print(f"  Batch {batch_idx+1}/{len(dataloader)} | Loss: {loss.item():.4f} | Running Avg Loss: {running_loss / (batch_idx + 1):.4f}")

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    print(f"\n Epoch {epoch+1} Finished - Avg Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")

print("\n Training Complete!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]


 Starting Training on cuda


 Epoch 1/10
  Batch 1/965 | Loss: 5.6721 | Running Avg Loss: 5.6721
  Batch 10/965 | Loss: 5.3863 | Running Avg Loss: 5.4764
  Batch 20/965 | Loss: 5.5479 | Running Avg Loss: 5.4523
  Batch 30/965 | Loss: 5.1680 | Running Avg Loss: 5.4020
  Batch 40/965 | Loss: 5.2316 | Running Avg Loss: 5.3657
  Batch 50/965 | Loss: 5.2881 | Running Avg Loss: 5.3495
  Batch 60/965 | Loss: 5.2898 | Running Avg Loss: 5.3381
  Batch 70/965 | Loss: 5.1480 | Running Avg Loss: 5.3256
  Batch 80/965 | Loss: 5.0583 | Running Avg Loss: 5.3114
  Batch 90/965 | Loss: 5.1052 | Running Avg Loss: 5.2956
  Batch 100/965 | Loss: 5.1065 | Running Avg Loss: 5.2751
  Batch 110/965 | Loss: 5.0802 | Running Avg Loss: 5.2514
  Batch 120/965 | Loss: 4.6832 | Running Avg Loss: 5.2262
  Batch 130/965 | Loss: 4.8985 | Running Avg Loss: 5.1924
  Batch 140/965 | Loss: 4.4874 | Running Avg Loss: 5.1648
  Batch 150/965 | Loss: 4.7027 | Running Avg Loss: 5.1303
  Batch 160/965 | Loss: 4.7051 | Running 

KeyboardInterrupt: 

# Evaluating the model

In [11]:
# top-1 accuracy, top-5 accuracy, and macro f1
from sklearn.metrics import accuracy_score, f1_score
import torch

model.eval()
all_preds = []
all_labels = []
top5_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        #Top-1 Predictions
        _, top1_preds = torch.max(outputs, 1)
        all_preds.extend(top1_preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # Top-5 Accuracy
        top5_preds = torch.topk(outputs, k=5, dim=1).indices  # Get top-5 indices
        top5_correct += torch.sum(top5_preds.eq(labels.view(-1, 1)))  # Count correct top-5
        total_samples += labels.size(0)

#Calculate Metrics
top1_accuracy = accuracy_score(all_labels, all_preds)
top5_accuracy = top5_correct.item() / total_samples
macro_f1 = f1_score(all_labels, all_preds, average='macro')

print(f"Top-1 Accuracy: {top1_accuracy:.4f}") # 60.45
print(f"Top-5 Accuracy: {top5_accuracy:.4f}") # 88.53
print(f"Macro F1 Score: {macro_f1:.4f}") # 59.03


Top-1 Accuracy: 0.6045
Top-5 Accuracy: 0.8853
Macro F1 Score: 0.5903


saving

In [12]:
torch.save(model.state_dict(), "vit_leafsnap.pth")